In [0]:
%run ./env

In [0]:
def mount_adls(blobContainerName):
    try:
        dbutils.fs.mount(
            source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, ACCOUNT_NAME),
            mount_point = f"/mnt/{ACCOUNT_NAME}/{blobContainerName}",
            extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + ACCOUNT_NAME + '.blob.core.windows.net': SAS_TOKEN}
        )
        print("OK!")
    except Exception as e:
        print("Falha", e)

OK!


In [0]:
mounts = dbutils.fs.mounts()

if not any(mount.mountPoint == f"/mnt/{ACCOUNT_NAME}/bronze" for mount in mounts):
    mount_adls("bronze")

if not any(mount.mountPoint == f"/mnt/{ACCOUNT_NAME}/silver" for mount in mounts):
    mount_adls("silver")

if not any(mount.mountPoint == f"/mnt/{ACCOUNT_NAME}/gold" for mount in mounts):
    mount_adls("gold")

In [0]:
import pandas as pd

jdbc_url = f"jdbc:sqlserver://{DB_SERVER}:1433;database={DB_DATABASE}"
connection_properties = {
  "user" : DB_USER,
  "password" : DB_PASS,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

query = f"(SELECT table_name FROM INFORMATION_SCHEMA.TABLES WHERE table_schema = '{DB_SCHEMA}') AS query"
df = spark.read.jdbc(url=jdbc_url, table=query, properties=connection_properties).toPandas()

# Para cada tabela encontrada, ler os dados e carregar para o Azure Data Lake Storage
for index, row in df.iterrows():
    table_name = row["table_name"]
    query2 = f"(SELECT * FROM {DB_SCHEMA}.{table_name}) as query"
    df_table = spark.read.jdbc(url=jdbc_url, table=query2, properties=connection_properties) 
    df_table.write \
        .format("com.databricks.spark.csv") \
        .option("header", "true") \
        .mode("overwrite") \
        .save(f"/mnt/{ACCOUNT_NAME}/landing/{table_name}.csv")
    print(f"Dados da tabela '{table_name}' carregados com sucesso.")